In [1]:
import os
import pandas as pd

In [2]:
df = pd.read_csv("chatbot-responses.csv")
df.head()

,conversation_id,human,bot
0,1,Hello,"Hi, am Kate. How can I help you today?"
1,2,Hi,"Hi, am Kate. How can I help you today?"
2,3,What's your name?,"My name is Kate, your helpful Kenyan travel as..."
3,4,Can you tell me about yourself?,"Of course! I'm Kate, a Kenyan bot specializing..."
4,5,Where are you from?,"I'm from Kenya, and I'm here to help you explo..."


In [3]:
os.environ["NEO4J_URI"] = "neo4j+s://2c883ba3.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "pCg79y7HP88UIzBBANW8GGz6xrn-nXi1GWzYb8EujrA"

In [4]:
from langchain_community.graphs import Neo4jGraph

In [5]:
graph = Neo4jGraph()

In [9]:
!neo4j config --database=neo4j set dbms.security.allow_csv_import_from_file_urls=true

Usage: neo4j { console | start | stop | restart | status | version }


In [ ]:
conversation_query = """
LOAD CSV WITH HEADERS FROM 'http://0.0.0.0:8000/chatbot-responses.csv' AS row
MERGE (c:Conversation {id: row.conversation_id})
SET c.question = row.human,
    c.response = row.bot
FOREACH (ignore IN CASE WHEN row.human IS NOT NULL THEN [1] ELSE [] END |
    MERGE (u:User {name: row.human})
    MERGE (b:Bot {name: row.bot})
    MERGE (u)-[:INTERACTS_WITH]->(c)
    MERGE (c)-[:RESPONDS_TO]->(b)
)
"""
graph.query(conversation_query);

In [ ]:
graph.refresh_schema(conversation_query)
print(graph.schema)